# Imports

In [3]:
import os
import warnings
import sys

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from sklearn import linear_model, preprocessing, metrics, model_selection
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from mlflow.tracking import MlflowClient

import seaborn as sn

if 'inline_rc' not in dir():
    inline_rc = dict(mpl.rcParams)

SEED = 10
np.random.seed(SEED)

In [4]:
mpl.rcParams.update(inline_rc)
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 14}
mpl.rc('font', **font)
lines = {'linewidth' : 3}
mpl.rc('lines', **lines)

In [5]:
registered_model_name = 'modelo_projeto_kobe'
min_precision = 0.7
model_version = -1 # recuperar a ultima versao
nexamples = 4

In [52]:
pred_holdout = pc.predict_model(bestmodel)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Decision Tree Classifier,0.5999,0.6010,0.3792,0.6113,0.4681,0.1747,0.1876


## Servir Dados do Modelo Kobe

In [73]:
from mlflow.tracking import MlflowClient
mlflow.set_registry_uri("sqlite:///mlruns.db")

registered_model_name = 'modelo_kobe'
min_precision = 0.7
model_version = -1 # recuperar a ultima versao
nexamples = 4

In [56]:
# Pycaret exporta junto o pipeline de preprocessamento
pc.save_model(bestmodel, f'./{registered_model_name}') 
# Carrega novamente o pipeline + bestmodel
model_pipe = pc.load_model(f'./{registered_model_name}')

Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Loaded


In [57]:
# Assinatura do Modelo Inferida pelo MLFlow
model_features = list(df_kobe_2p.drop(kobe_target_col, axis=1).columns)
inf_signature = infer_signature(df_kobe_2p[model_features], model_pipe.predict(df_kobe_2p))
# Exemplo de entrada para o MLmodel
input_example = {x: df_kobe_2p[x].values[:nexamples] for x in model_features}
# Log do pipeline de modelagem do sklearn e registrar como uma nova versao

In [58]:
# Log do pipeline de modelagem do sklearn e registrar como uma nova versao
mlflow.sklearn.log_model(
     sk_model=model_pipe,
     artifact_path="sklearn-model",
     registered_model_name=registered_model_name,
     signature = inf_signature,
     input_example = input_example
 )
# Criacao do cliente do servico MLFlow e atualizacao versao modelo
client = MlflowClient()
if model_version == -1:
    model_version = client.get_latest_versions(registered_model_name)[-1].version
# Registrar o modelo como staging
client.transition_model_version_stage(
     name=registered_model_name,
     version=model_version, # Verificar com usuario qual versao
     stage="Staging"
)

Registered model 'modelo_kobe' already exists. Creating a new version of this model...
2022/09/05 18:44:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: modelo_kobe, version 3
Created version '3' of model 'modelo_kobe'.


<ModelVersion: creation_timestamp=1662414278222, current_stage='Staging', description=None, last_updated_timestamp=1662414278285, name='modelo_kobe', run_id='fec5b1b4be984cdf8e96293b56fc02d9', run_link=None, source='./mlruns/2/fec5b1b4be984cdf8e96293b56fc02d9/artifacts/sklearn-model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [59]:
mlflow.end_run()

In [80]:
import os
os.environ['MLFLOW_TRACKING_URI'] = 'sqlite:///mlruns.db'
!mlflow models serve -m "models:/modelo_kobe/Staging" --no-conda -p 5002

^C


In [81]:
import requests
host = 'localhost'
port = '5002'
url = f'http://{host}:{port}/invocations'
headers = {'Content-Type': 'application/json'}

http_data = data_operation.drop(kobe_target_col,axis=1).to_json(orient='split')
r = requests.post(url=url, headers=headers, data=http_data)

In [62]:
data_operation.loc[:, 'operation_label'] = pd.read_json(r.text).values[:,0]

In [63]:
data_operation.head(5)

,lat,lon,minutes_remaining,period,playoffs,shot_distance,shot_made_flag,operation_label
24824,33.9913,-118.3178,8,4,0,7,1.0,0
10535,33.9703,-118.2908,11,3,0,7,1.0,1
26057,33.8863,-118.3848,6,3,0,19,1.0,0
27642,34.0653,-118.3818,7,3,1,11,0.0,0
30391,33.9913,-118.3658,6,4,1,10,1.0,0


In [64]:
print(metrics.classification_report(data_operation[kobe_target_col], data_operation['operation_label']))

              precision    recall  f1-score   support

         0.0       0.58      0.65      0.61      2133
         1.0       0.55      0.49      0.52      1924

    accuracy                           0.57      4057
   macro avg       0.57      0.57      0.57      4057
weighted avg       0.57      0.57      0.57      4057



## Consumir o Serviço do Kobe Bryant

In [65]:
!curl http://localhost:5002/invocations -X POST -H "Content-Type:application/json; format=pandas-split" --data "{\"columns\":[\"lat\", \"lon\", \"minutes_remaining\", \"period\",\"playoffs\",\"shot_distance\"],\"data\":[[35.9913,-120.3178,8,4,0,7],[33.9703,-118.2908,11,3,0,7]]}"

[0, 1]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   156  100     6  100   150    161   4045 --:--:-- --:--:-- --:--:--  4333


In [69]:
df_kobe_3p['operation_label'].value_counts()

0    5411
1       1
Name: operation_label, dtype: int64

In [70]:
df_kobe_3p[kobe_target_col].value_counts()

0.0    3630
1.0    1782
Name: shot_made_flag, dtype: int64

In [67]:
http_data = df_kobe_3p.drop(kobe_target_col,axis=1).to_json(orient='split')
r = requests.post(url=url, headers=headers, data=http_data)

In [68]:
df_kobe_3p.loc[:, 'operation_label'] = pd.read_json(r.text).values[:,0]

print(metrics.classification_report(df_kobe_3p[kobe_target_col], df_kobe_3p['operation_label']))

              precision    recall  f1-score   support

         0.0       0.67      1.00      0.80      3630
         1.0       0.00      0.00      0.00      1782

    accuracy                           0.67      5412
   macro avg       0.34      0.50      0.40      5412
weighted avg       0.45      0.67      0.54      5412



In [83]:
import os
os.environ['MLFLOW_TRACKING_URI'] = 'sqlite:///mlruns.db'
!mlflow models serve -m "models:/modelo_kobe/Staging" --no-conda -p 5003

^C


In [84]:
import requests
host = 'localhost'
port = '5003'
url = f'http://{host}:{port}/invocations'
headers = {'Content-Type': 'application/json'}

http_data = data_operation.drop(kobe_target_col,axis=1).to_json(orient='split')
r = requests.post(url=url, headers=headers, data=http_data)

In [85]:
data_operation.loc[:, 'operation_label'] = pd.read_json(r.text).values[:,0]

In [88]:
!curl http://localhost:5003/invocations -X POST -H "Content-Type:application/json; format=pandas-split" --data "{\"columns\":[\"lat\", \"lon\", \"minutes_remaining\", \"period\",\"playoffs\",\"shot_distance\"],\"data\":[[35.9913,-120.3178,8,4,0,7],[33.9703,-118.2908,11,3,0,7]]}"

[0, 0]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   156  100     6  100   150    130   3272 --:--:-- --:--:-- --:--:--  3466


In [89]:
http_data = df_kobe_3p.drop(kobe_target_col,axis=1).to_json(orient='split')
r = requests.post(url=url, headers=headers, data=http_data)

In [90]:
df_kobe_3p.loc[:, 'operation_label'] = pd.read_json(r.text).values[:,0]

print(metrics.classification_report(df_kobe_3p[kobe_target_col], df_kobe_3p['operation_label']))

              precision    recall  f1-score   support

         0.0       0.67      0.94      0.78      3630
         1.0       0.35      0.07      0.12      1782

    accuracy                           0.65      5412
   macro avg       0.51      0.50      0.45      5412
weighted avg       0.56      0.65      0.56      5412



In [92]:
df_kobe_3p['operation_label'].value_counts()

0    5057
1     355
Name: operation_label, dtype: int64